# Signal

Various utilities to do signal filtering and quality checking.

In [ ]:
# Imports
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from eegclassify import main, load, clean, features, preprocess, transform
from eegwatch.devices.muse import CHANNELS_MUSE

%matplotlib inline
plt.rcParams['figure.figsize'] = [15, 5]

N_SAMPLES_PLOT = 400

In [ ]:
%%javascript
document.title='erb-thesis/Signal - Jupyter'  // Set the document title to be able to track time spent working on the notebook with ActivityWatch

In [ ]:
# Pick the last recording
files = load._get_all_recording_files()
files = sorted(files)[-1:]

# Load the last files
df = load.load_eeg(files)
X = df.drop(columns=['timestamp']).to_numpy()
print(X.shape)

In [ ]:
ax = sns.lineplot(data=X[-N_SAMPLES_PLOT:, :], dashes=False)
ax.set_title("Raw signal (pre-filtering)")
ax.legend(labels=CHANNELS_MUSE);

In [ ]:
X_f = clean.filter(X)
ax = sns.lineplot(data=X_f[-N_SAMPLES_PLOT:, :], dashes=False)
ax.set_title("Filtered signal")
ax.legend(labels=CHANNELS_MUSE);

In [ ]:
# Check the stddev of each channel
# TODO: refactor signal check function to actually use this
X_check = X_f[-N_SAMPLES_PLOT:, :]
print("mean\t", np.mean(X_check, axis=0))
print("std\t", np.std(X_check, axis=0))
print("maxabs\t", np.max(np.abs(X_check), axis=0))